### Configuration Requirements:


Definition of Environment Variables
- OPENAI_API_KEY= [openai api key]
- OPENAI_ORGANIZATION=[organization key]
- LANGCHAIN_TRACING_V2=true
- LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
- LANGCHAIN_API_KEY=[ langsmith apikey]
- LANGCHAIN_PROJECT=[project name]


In [ ]:
# !pip install -U langchain langsmith



In [ ]:
from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

### Tracing

#### Trace Example 1

In [ ]:
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable

client = wrap_openai(openai.Client())

@traceable
def get_response(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-3.5-turbo"
    )
    return result.choices[0].message.content

response = get_response("What is LLMops in Machine Learning? What are the best tools for LLMops operation?")


In [ ]:
print(response)

#### Trace Example 2

In [ ]:
import openai
from langsmith import traceable

client = wrap_openai(openai.Client())

@traceable
def openai_response(system_prompt: str, user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user","content":user_input}],
        model="gpt-3.5-turbo",
        temperature=0.5,
        frequency_penalty=0.1,
        timeout=5,

    )
    return result.choices[0].message.content



In [ ]:
system_prompt = '''You are an eloquent travel assistant. Your task is to recommend best cities for a vacation to users based on the requirements of the user. Understand the requirements and also recommend the things users need to carry with them to travel to the given cities.'''
user_prompt = '''I want to travel in February. The city must not be freezing cold neither too hot to travel. Asian Cities will be preferred.'''
response = openai_response(system_prompt,user_prompt)

In [ ]:
print(response)

#### Trace Example 3

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

model = ChatOpenAI(temperature=0)


joke_query = "Tell me a joke."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

### Dataset Creation

Methods of Dataset Creation:
1. Importing CSV files
2. Through Existing Runs

In [ ]:
from langsmith import Client

example_inputs = [
  ("What is the largest mammal?", "The blue whale"),
  ("What do mammals and birds have in common?", "They are both warm-blooded"),
  ("What are reptiles known for?", "Having scales"),
  ("What's the main characteristic of amphibians?", "They live both in water and on land"),
]

client = Client()
dataset_name = "Elementary Animal Questions"

dataset = client.create_dataset(
    dataset_name=dataset_name, description="Questions and answers about animal phylogenetics.",
)
for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        metadata={"source": "Wikipedia"},
        dataset_id=dataset.id,
    )

In [ ]:
import os
from langsmith import Client

client = Client()
dataset_name = "run1"

runs = client.list_runs(
    project_name=os.environ.get('LANGCHAIN_PROJECT'),
    execution_order=1,
    error=False,
)

dataset = client.create_dataset(dataset_name, description="Dataset creation using Runs")
for run in runs:
    client.create_example(
        inputs=run.inputs,
        outputs=run.outputs,
        dataset_id=dataset.id,
    )

### Evaluation

#### Evaluation by Comparison

In [ ]:
from langsmith import Client
from langchain_openai import chat_models
from langchain import prompts, smith
from langchain.schema import output_parser


# Define your runnable or chain below.
system_prompt = """You are an eloquent travel assistant. Your task is to recommend best cities for a vacation to users based on the requirements of the user. Understand the requirements and also recommend the things users need to carry with them to travel to the given cities."""
user_prompt = """I want to travel in February. The city must not be freezing cold neither too hot to travel. Asian Cities will be preferred."""
prompt = prompts.ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{user_input}")]
)
llm = chat_models.ChatOpenAI(model="gpt-4", temperature=0)
chain = prompt | llm | output_parser.StrOutputParser()

client = Client()


results = client.run_on_dataset(
    dataset_name="langsmith_dataset",
    llm_or_chain_factory=chain,
    project_name="langsmith_eval5",
)

#### Evaluation by LangSmith Evaluators

##### Evaluation in Chat model

In [ ]:
examples = [
    {
        "inputs": {
            "question": "How does that apply?",
            "chat_history": [
                {
                    "role": "user",
                    "content": "I’m trying tto uunderstand instein’s theory.",
                },
                {
                    "role": "assistant",
                    "content": "Which one? He's known for several theories.",
                },
                {"role": "user", "content": "Thee one about time and spae."},
                {
                    "role": "assistant",
                    "content": "Ah, you're referring to the theory of relativity. There are two parts: special and general. Which one?",
                },
            ],
        },
        "outputs": {
            "expected": "Special relativity, proposed by Einstein in 1905, deals with objects in uniform motion, especially those moving at the speed of light. It introduced the idea that time and space are relative and can change in relation to each other. For instance, time can appear to move slower for an object moving close to the speed of light."
        },
    },
    {
        "inputs": {
            "question": "What's the main difference?",
            "chat_history": [
                {"role": "user", "content": "Can yyou contrast DNA and RNA for me?"},
                {
                    "role": "assistant",
                    "content": "Certainly. DNA and RNA are both nucleic acids but have different roles, structures, and properties. Do you want specifics?",
                },
            ],
        },
        "outputs": {
            "expected": "The main structural differences between DNA and RNA include: 1) DNA is double-stranded while RNA is single-stranded. 2) The sugar in the backbone of RNA is ribose, whereas in DNA it's deoxyribose. 3) DNA uses the bases adenine (A), cytosine (C), guanine (G), and thymine (T); RNA uses adenine (A), cytosine (C), guanine (G), and uracil (U) instead of thymine."
        },
    },
    {
        "inputs": {
            "question": "what led them to such a draastic action?",
            "chat_history": [
                {"role": "user", "content": "tell me about the Boston Tea Party."},
                {
                    "role": "assistant",
                    "content": "The Boston Tea Party was a political protest by the American colonists against the British government in 1773. They were protesting the Tea Act, which allowed the British East India Company to sell tea directly to the colonies, bypassing colonial merchants.",
                },
            ],
        },
        "outputs": {
            "expected": "The colonists undertook the Boston Tea Party as a drastic action due to multiple reasons: 1) They believed the Tea Act was a violation of their rights as Englishmen, as they were being taxed without their consent. 2) The act gave the British East India Company a monopoly on tea sales, threatening local businesses. 3) The act was seen as another example of the British government's increasing interference in colonial affairs. The protest was a way to show their strong opposition to British policies."
        },
    },
    {
        "inputs": {
            "question": "thats a scary one. can it be avoideed?",
            "chat_history": [
                {"role": "user", "content": "I'm learning bout genetic disorders."},
                {
                    "role": "assistant",
                    "content": "Genetic disorders are diseases caused by abnormalities in an individual's DNA. They can be inherited or result from mutations. One common one is Huntington's disease.",
                },
            ],
        },
        "outputs": {
            "expected": "Huntington's disease is a hereditary genetic disorder caused by a mutation in the HTT gene. If a person inherits the defective gene, they will eventually develop the disease. Currently, there's no cure for Huntington's, but its onset can be delayed with treatment. Genetic counseling and testing can help prospective parents understand the risks of passing the mutation to their offspring."
        },
    },
    {
        "inputs": {
            "question": "Which one?",
            "chat_history": [
                {
                    "role": "user",
                    "content": "I'm confused aboutt stars. what even aaaare they?",
                },
                {
                    "role": "assistant",
                    "content": "Stars are celestial bodies made mostly of hydrogen and helium. They generate light and heat through nuclear fusion in their cores.",
                },
                {
                    "role": "user",
                    "content": "there''s a classification based on theirbrightness, right?",
                },
                {
                    "role": "assistant",
                    "content": "Yes",
                },
            ],
        },
        "outputs": {
            "expected": "Yes, stars are classified based on their brightness using a system called the Hertzsprung-Russell (H-R) diagram. In this diagram, stars are categorized into main-sequence stars, giants, supergiants, and white dwarfs, based on their luminosity and temperature. The Sun, for instance, is a main-sequence star."
        },
    },
]

In [ ]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Chat Bot Evals Single-Turn Example - {uuid.uuid4()}"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    inputs=[e["inputs"] for e in examples],
    outputs=[e["outputs"] for e in examples],
    dataset_id=dataset.id,
)

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

# An example chain
chain = (
    ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful tutor AI."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
        ]
    )
    | ChatOpenAI(model="gpt-3.5-turbo")
    | StrOutputParser()
)

In [ ]:
from langchain.adapters.openai import convert_openai_messages
from langsmith.evaluation import LangChainStringEvaluator, evaluate
from langsmith.schemas import Example, Run


def predict(inputs: dict):
    return chain.invoke(
        {
            "input": inputs["question"],
            "chat_history": convert_openai_messages(inputs["chat_history"]),
        }
    )


def format_evaluator_inputs(run: Run, example: Example):
    return {
        "input": example.inputs["question"],
        "prediction": next(iter(run.outputs.values())),
        "reference": example.outputs["expected"],
    }


correctness_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={"criteria": "correctness", "normalize_by": 10},
    prepare_data=format_evaluator_inputs,
)

results = evaluate(
    predict,
    data=dataset_name,
    experiment_prefix="Chat Single Turn",
    evaluators=[correctness_evaluator],
    metadata={"model": "gpt-3.5-turbo"},
)

##### Evaluation from Run Dataset

In [ ]:
from langsmith import Client
from langchain_openai.chat_models import ChatOpenAI
from langchain.smith import RunEvalConfig, run_on_dataset
from langchain import prompts,smith
from langchain.schema import output_parser

system_prompt = '''You are an eloquent travel assistant. Your task is to recommend best cities for a vacation to users based on the requirements of the user. Understand the requirements and also recommend the things users need to carry with them to travel to the given cities.'''
user_prompt = '''I want to travel in February. The city must not be freezing cold neither too hot to travel. Asian Cities will be preferred.'''
prompt = prompts.ChatPromptTemplate.from_messages(
  [
    ("system", system_prompt),
    ("human", "{user_input}")
  ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = prompt | llm | output_parser.StrOutputParser()

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa",
    ],
    reference_key="output",
    input_key="user_input",
)

client = Client()
run_on_dataset(
    client=client,
    dataset_name="langsmith_dataset",
    llm_or_chain_factory=chain,
    evaluation=evaluation_config,
)

### Threads

In [ ]:
q1 = "What is LLMOps in Machine Learning? What are the best tools for LLMOps operation?"

q2 = "What is the difference between MLOps and LLMOps in Machine Learning"


In [ ]:
import openai
from langsmith import traceable
import uuid

client = openai.Client()
# define session id
session_id = str(uuid.uuid4())

# define function for call
@traceable(
    run_type="chain",
    name="OpenAI Assistant",
)
def assistant(messages: list[dict]):
    return (
        client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": "You are a Machine Learning assistant specialized in MLOps."}]
            + messages,
        )
        .choices[0]
        .message
    )


messages = [{"role": "user", "content": q1}]
response = assistant(messages, langsmith_extra={"metadata": {"session_id": session_id}})
messages = messages + [response, {"role": "user", "content": q2}]
response = assistant(messages, langsmith_extra={"metadata": {"session_id": session_id}})